In [1]:
import tensorflow as tf
import image_from_tfrecord as read_data
import time
import numpy as np

print(tf.__version__)

1.1.0


In [2]:
LOAD_MODEL = False
START_ITER = 0
model_path = "./model-0000.ckpt"

# Define Prameters
DATA_TYPE = tf.float32
SEED = 4658

NUM_QUEUE_THREAD = 4

BATCH_SIZE_TRAIN = 100
BATCH_SIZE_VALIDATION = 25

IMAGE_HEIGHT = 180
IMAGE_WIDTH = 140
NUM_CHANNEL = 3

NUM_TRAIN_CASE = 193515
NUM_VALIDATION_CASE = 48379
NUM_CLASS = 8277

WEIGHT_DECAY_FACTOR = 0.0005
MOMENTUM_FACTOR = 0.9

NUM_EPOCH = 30
SAVE_FREQ = 5000
TEST_FREQ = 1000
DISP_FREQ = 10
MAX_ITER = 100 # 400000

MAX_ITER_TEST = NUM_VALIDATION_CASE // BATCH_SIZE_VALIDATION

BASE_LR = 0.007
regularizer = tf.contrib.layers.l2_regularizer(scale=WEIGHT_DECAY_FACTOR)

In [3]:
# # Define Weight Variables
# with tf.variable_scope("layer1"):
#     conv11_weight =  tf.Variable(
#         tf.random_normal([3, 3, NUM_CHANNEL, 64],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='conv1')
#     conv11_bias = tf.Variable(
#         tf.constant(0, shape=[64],
#                     dtype=DATA_TYPE), name='bias1')
#     conv12_weight = tf.Variable(
#         tf.random_normal([3, 3, 64, 64],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight2')
#     conv12_bias = tf.Variable(
#         tf.constant(0, shape=[64],
#                     dtype=DATA_TYPE), name='bias2')
# with tf.variable_scope("layer2"):
#     conv21_weight = tf.Variable(
#         tf.random_normal([3, 3, 64, 128],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight1')
#     conv21_bias = tf.Variable(
#         tf.constant(0, shape=[128],
#                     dtype=DATA_TYPE), name='bias1')
#     conv22_weight = tf.Variable(
#         tf.random_normal([3, 3, 128, 128],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight2')
#     conv22_bias = tf.Variable(
#         tf.constant(0, shape=[128],
#                     dtype=DATA_TYPE), name='bias2')
# with tf.variable_scope("layer3"):
#     conv31_weight = tf.Variable(
#         tf.random_normal([3, 3, 128, 256],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight1')
#     conv31_bias = tf.Variable(
#         tf.constant(0, shape=[256],
#                     dtype=DATA_TYPE), name='bias1')
#     conv32_weight = tf.Variable(
#         tf.random_normal([3, 3, 256, 256],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight2')
#     conv32_bias = tf.Variable(
#         tf.constant(0, shape=[256],
#                     dtype=DATA_TYPE), name='bias2')
# with tf.variable_scope("layer4"):
#     conv41_weight = tf.Variable(
#         tf.random_normal([3, 3, 256, 512],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight1')
#     conv41_bias = tf.Variable(
#         tf.constant(0, shape=[512],
#                     dtype=DATA_TYPE), name='bias1')
#     conv42_weight = tf.Variable(
#         tf.random_normal([3, 3, 512, 512],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight2')
#     conv42_bias = tf.Variable(
#         tf.constant(0, shape=[512],
#                     dtype=DATA_TYPE), name='bias2')
#     conv43_weight = tf.Variable(
#         tf.random_normal([3, 3, 512, 512],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight3')
#     conv43_bias = tf.Variable(
#         tf.constant(0, shape=[512],
#                     dtype=DATA_TYPE), name='bias3')
# with tf.variable_scope("layer5"):
#     conv51_weight = tf.Variable(
#         tf.random_normal([3, 3, 512, 512],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight1')
#     conv51_bias = tf.Variable(
#         tf.constant(0, shape=[512],
#                     dtype=DATA_TYPE), name='bias1')
#     conv52_weight = tf.Variable(
#         tf.random_normal([3, 3, 512, 512],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight2')
#     conv52_bias = tf.Variable(
#         tf.constant(0, shape=[512],
#                     dtype=DATA_TYPE), name='bias2')
#     conv53_weight = tf.Variable(
#         tf.random_normal([3, 3, 512, 512],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight3')
#     conv53_bias = tf.Variable(
#         tf.constant(0, shape=[512],
#                     dtype=DATA_TYPE), name='bias3')
# with tf.variable_scope("layer6"):
#     fc6_weight = tf.Variable(
#         tf.random_normal([10240, 4096],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight')
#     fc6_bias = tf.Variable(
#         tf.constant(0, shape=[4096],
#                     dtype=DATA_TYPE), name='bias')
# with tf.variable_scope("layer7"):
#     fc7_weight = tf.Variable(
#         tf.random_normal([4096, 4096],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight')
#     fc7_bias = tf.Variable(
#         tf.constant(0, shape=[4096],
#                     dtype=DATA_TYPE), name='bias')
# with tf.variable_scope("layer8"):
#     fc8_weight = tf.Variable(
#         tf.random_normal([4096, NUM_CLASS + 1],  # [Kernel_H, kernel_W, NUM_IN, NUM_OUT]
#                          stddev=0.01, seed=SEED, dtype=DATA_TYPE), name='weight')
#     fc8_bias = tf.Variable(
#         tf.constant(0, shape=[NUM_CLASS + 1],
#                     dtype=DATA_TYPE), name='bias')

In [4]:
with tf.variable_scope("layer1"):
    conv11_weight = tf.get_variable("weight1", shape=[3, 3, NUM_CHANNEL, 64],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv11_bias = tf.Variable(
        tf.constant(0, shape=[64],
                    dtype=DATA_TYPE), name='bias1')
    conv12_weight = tf.get_variable("weight2", shape=[3, 3, 64, 64],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv12_bias = tf.Variable(
        tf.constant(0, shape=[64],
                    dtype=DATA_TYPE), name='bias2')
with tf.variable_scope("layer2"):
    conv21_weight = tf.get_variable("weight1", shape=[3, 3, 64, 128],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv21_bias = tf.Variable(
        tf.constant(0, shape=[128],
                    dtype=DATA_TYPE), name='bias1')
    conv22_weight = tf.get_variable("weight2", shape=[3, 3, 128, 128],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv22_bias = tf.Variable(
        tf.constant(0, shape=[128],
                    dtype=DATA_TYPE), name='bias2')
with tf.variable_scope("layer3"):
    conv31_weight = tf.get_variable("weight1", shape=[3, 3, 128, 256],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv31_bias = tf.Variable(
        tf.constant(0, shape=[256],
                    dtype=DATA_TYPE), name='bias1')
    conv32_weight = tf.get_variable("weight2", shape=[3, 3, 256, 256],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv32_bias = tf.Variable(
        tf.constant(0, shape=[256],
                    dtype=DATA_TYPE), name='bias2')
with tf.variable_scope("layer4"):
    conv41_weight = tf.get_variable("weight1", shape=[3, 3, 256, 512],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv41_bias = tf.Variable(
        tf.constant(0, shape=[512],
                    dtype=DATA_TYPE), name='bias1')
    conv42_weight = tf.get_variable("weight2", shape=[3, 3, 512, 512],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv42_bias = tf.Variable(
        tf.constant(0, shape=[512],
                    dtype=DATA_TYPE), name='bias2')
    conv43_weight = tf.get_variable("weight3", shape=[3, 3, 512, 512],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv43_bias = tf.Variable(
        tf.constant(0, shape=[512],
                    dtype=DATA_TYPE), name='bias3')
with tf.variable_scope("layer5"):
    conv51_weight = tf.get_variable("weight1", shape=[3, 3, 512, 512],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv51_bias = tf.Variable(
        tf.constant(0, shape=[512],
                    dtype=DATA_TYPE), name='bias1')
    conv52_weight = tf.get_variable("weight2", shape=[3, 3, 512, 512],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED), dtype=DATA_TYPE)
    conv52_bias = tf.Variable(
        tf.constant(0, shape=[512],
                    dtype=DATA_TYPE), name='bias2')
    conv53_weight = tf.get_variable("weight3", shape=[3, 3, 512, 512],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    conv53_bias = tf.Variable(
        tf.constant(0, shape=[512],
                    dtype=DATA_TYPE), name='bias3')
with tf.variable_scope("layer6"):
    fc6_weight = tf.get_variable("weight", shape=[10240, 4096],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    fc6_bias = tf.Variable(
        tf.constant(0, shape=[4096],
                    dtype=DATA_TYPE), name='bias')
with tf.variable_scope("layer7"):
    fc7_weight = tf.get_variable("weight", shape=[4096, 4096],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    fc7_bias = tf.Variable(
        tf.constant(0, shape=[4096],
                    dtype=DATA_TYPE), name='bias')
with tf.variable_scope("layer8"):
    fc8_weight = tf.get_variable("weight", shape=[4096, NUM_CLASS + 1],
                                    initializer=tf.contrib.layers.xavier_initializer(seed=SEED),
                                    regularizer=regularizer, dtype=DATA_TYPE)
    fc8_bias = tf.Variable(
        tf.constant(0, shape=[NUM_CLASS + 1],
                    dtype=DATA_TYPE), name='bias')


In [5]:
#Define Deep Model
def model(input, dropout_prop):
    #Layer1
    conv = tf.nn.conv2d(input, conv11_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv11_bias)
    relu = tf.nn.relu(conv)
    conv = tf.nn.conv2d(relu, conv12_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv12_bias)
    relu = tf.nn.relu(conv)
    pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    # Layer2
    conv = tf.nn.conv2d(pool, conv21_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv21_bias)
    relu = tf.nn.relu(conv)
    conv = tf.nn.conv2d(relu, conv22_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv22_bias)
    relu = tf.nn.relu(conv)
    pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    # Layer3
    conv = tf.nn.conv2d(pool, conv31_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv31_bias)
    relu = tf.nn.relu(conv)
    conv = tf.nn.conv2d(relu, conv32_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv32_bias)
    relu = tf.nn.relu(conv)
    pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    # Layer4
    conv = tf.nn.conv2d(pool, conv41_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv41_bias)
    relu = tf.nn.relu(conv)
    conv = tf.nn.conv2d(relu, conv42_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv42_bias)
    relu = tf.nn.relu(conv)
    conv = tf.nn.conv2d(relu, conv43_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv43_bias)
    relu = tf.nn.relu(conv)
    pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    # Layer5
    conv = tf.nn.conv2d(pool, conv51_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv51_bias)
    relu = tf.nn.relu(conv)
    conv = tf.nn.conv2d(relu, conv52_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv52_bias)
    relu = tf.nn.relu(conv)
    conv = tf.nn.conv2d(relu, conv53_weight, strides=[1, 1, 1, 1], padding="SAME")
    conv = tf.nn.bias_add(conv, conv53_bias)
    relu = tf.nn.relu(conv)
    pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    #Layer6
    pool_shape = pool.get_shape().as_list()
    pool_reshape = tf.reshape(pool, [-1, pool_shape[1] * pool_shape[2] * pool_shape[3]])
    ip = tf.matmul(pool_reshape, fc6_weight)
    ip = ip + fc6_bias
    relu = tf.nn.relu(ip)
    do = tf.nn.dropout(relu, dropout_prop, seed=SEED)
    # Layer7
    ip = tf.matmul(do, fc7_weight)
    ip = ip + fc7_bias
    relu = tf.nn.relu(ip)
    do = tf.nn.dropout(relu, dropout_prop, seed=SEED)
    # Layer7
    ip = tf.matmul(do, fc8_weight)
    ip = ip + fc8_bias

    return ip

In [6]:
tfrecords_filename_train = '../../../DB/data/train_mirror.tfrecords'
filename_queue_train = tf.train.string_input_producer([tfrecords_filename_train])
image_train, annotation_train = read_data.read_and_decode(filename_queue_train, IMAGE_HEIGHT, IMAGE_WIDTH,
                                                          NUM_CHANNEL, NUM_CLASS, BATCH_SIZE_TRAIN, NUM_QUEUE_THREAD)

tfrecords_filename_test = '../../../DB/data/validation.tfrecords'
filename_queue_test = tf.train.string_input_producer([tfrecords_filename_test])
image_test, annotation_test = read_data.read_and_decode(filename_queue_test, IMAGE_HEIGHT, IMAGE_WIDTH,
                                                        NUM_CHANNEL, NUM_CLASS, BATCH_SIZE_VALIDATION, NUM_QUEUE_THREAD)

print(image_train.shape)
print(image_test.shape)

(100, 180, 140, 3)
(25, 180, 140, 3)


In [7]:
# Calculate loss
data = tf.placeholder(
      DATA_TYPE,
      [None, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNEL])
label = tf.placeholder(tf.int32, [None, NUM_CLASS + 1])
do_prop = tf.placeholder(tf.float32)
logit = model(data, do_prop)

In [8]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=logit))
# Weight decay term
reg_ws = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss += tf.reduce_sum(reg_ws)
# variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
# for variable in variables:
#     loss += WEIGHT_DECAY_FACTOR * tf.sqrt(tf.reduce_sum(tf.square(variable)))

# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
global_step = tf.Variable(0, dtype=DATA_TYPE)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.polynomial_decay(
    learning_rate=BASE_LR,
    global_step=global_step,
    decay_steps=MAX_ITER * 2,
    end_learning_rate=0.0,
    power=1.0
)

In [9]:
# Train operator
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=MOMENTUM_FACTOR)\
    .minimize(loss, global_step=global_step)

# Calculate accuracy
prediction = tf.argmax(tf.nn.softmax(logit), 1)
train_label = tf.argmax(label, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, train_label), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
# init_op = tf.initialize_all_variables()
saver = tf.train.Saver()

total_mean = np.zeros([1, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNEL], dtype=np.float32)
with open("image_mean.txt", "r") as fd:
    image_list = fd.read()
    image_list = image_list.split("\n")
    idx = 0
    for i in range(IMAGE_HEIGHT):
        for j in range(IMAGE_WIDTH):
            for k in range(NUM_CHANNEL):
                total_mean[0][i][j][k] = image_list[idx]
                idx += 1
total_mean_repeat_train = np.repeat(total_mean, BATCH_SIZE_TRAIN, axis=0)
total_mean_repeat_test = np.repeat(total_mean, BATCH_SIZE_VALIDATION, axis=0)
print(total_mean_repeat_train.shape)
print(total_mean_repeat_test.shape)

(100, 180, 140, 3)
(25, 180, 140, 3)


In [10]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    if LOAD_MODEL:
        saver.restore(sess, model_path)
    else:
        sess.run(init_op)

    for train_iter in range(START_ITER, MAX_ITER):
        # time_s = time.time()
        images_train, annotations_train = sess.run([image_train, annotation_train])
        images_train -= total_mean_repeat_train
        # time_e = time.time()
        # print("elapsed time for load image =", time_e - time_s, "(s)")

        time_s = time.time()
        if train_iter % DISP_FREQ == 0:
            #########################Disply Train Info.#################################
            print("iter", train_iter)
            _, loss_value, acc_value, learning_rate_value = \
                sess.run([optimizer, loss, accuracy, learning_rate],
                         feed_dict={data: images_train, label: annotations_train, do_prop: 0.5})
            time_e = time.time()

            # writer.add_summary(summary, global_step=train_iter)

            print("elapsed time for train=", time_e - time_s, "(s)")
            print("learning rate =", learning_rate_value)
            print("loss =", loss_value)
            print("acc =", acc_value)

            with open("./result.txt", "a") as fd:
                fd.write("%d, %f, %f, %f\n" % (train_iter, learning_rate_value, loss_value, acc_value))

            #########################Disply Train Info.#################################
            if train_iter % SAVE_FREQ == 0:
                saver.save(sess, "./model-" + str(train_iter) + ".ckpt")
                print("model-" + str(train_iter) + ".ckpt is saved!!")
        else:
            sess.run(optimizer,
                         feed_dict={data: images_train, label: annotations_train, do_prop: 0.5})
            # time_e = time.time()
            # print("elapsed time for train=", time_e - time_s, "(s)")

        if train_iter % TEST_FREQ == 0:
            acc_test_value = 0
            for test_iter in range(MAX_ITER_TEST):
                if test_iter % DISP_FREQ == 0:
                    print("iter test =", test_iter)
                images_test, annotations_test = sess.run([image_test, annotation_test])
                images_test -= total_mean_repeat_test
                acc_test_value += sess.run(accuracy, feed_dict={data: images_test, label: annotations_test, do_prop: 1.0})
            acc_test_value /= MAX_ITER_TEST

            print("*****************ACC for TEST***********************")
            print("acc for test = ", acc_test_value)
            print("****************************************************")
            with open("./acc_test.txt", "a") as fd_test_ret:
                fd_test_ret.write("%d, %f, %f\n" % (train_iter, learning_rate_value, acc_test_value))
                fd_test_ret.write("\n")

    coord.request_stop()
    coord.join(threads)

fd.close()

iter 0
elapsed time for train= 9.339600563049316 (s)
learning rate = 0.007
loss = 13.6799
acc = 0.0
model-0.ckpt is saved!!
iter test = 0
iter test = 10
iter test = 20
iter test = 30
iter test = 40
iter test = 50
iter test = 60
iter test = 70
iter test = 80
iter test = 90
iter test = 100
iter test = 110
iter test = 120
iter test = 130
iter test = 140
iter test = 150
iter test = 160
iter test = 170
iter test = 180
iter test = 190
iter test = 200
iter test = 210
iter test = 220
iter test = 230
iter test = 240
iter test = 250
iter test = 260
iter test = 270
iter test = 280
iter test = 290
iter test = 300
iter test = 310
iter test = 320
iter test = 330
iter test = 340
iter test = 350
iter test = 360
iter test = 370
iter test = 380
iter test = 390
iter test = 400
iter test = 410
iter test = 420
iter test = 430
iter test = 440
iter test = 450
iter test = 460
iter test = 470
iter test = 480
iter test = 490
iter test = 500
iter test = 510
iter test = 520
iter test = 530
iter test = 540
iter te